![DLI Header](images/DLI_Header.png)

# 星辰自行车 AI 助手
｜Star Bikes AI Assistant

在此笔记本中，您将创建一个 AI 助手，以帮助客户就从星辰自行车购买新自行车做出最佳决策。您还将简要了解您正在使用的模型的**令牌限制（token limits）** 及其对保留对话历史记录的影响。

## 学习目标
｜Learning Objectives

在完成此笔记本时，您将能够：
- 解释**令牌限制** 及其对 LLM 行为的影响。
- 构建一个能够（有限）记忆对话的 AI 助手，该助手不受设置的**令牌限制** 的影响。

## 视频演练

执行下面的单元格以加载此笔记本的视频演练。

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/07-assistant.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 管道
｜Create LLaMA-2 Pipeline

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"
# model = "TheBloke/Llama-2-7B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 获取 LLaMA-2 分词器
｜Get LLaMA-2 Tokenizer

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model)

## 辅助函数和类
｜Helper Functions and Classes

在此笔记本中，我们将使用以下函数和类来支持我们与 LLM 的交互。现在可以随意浏览一下，因为在下面使用时会更详细地介绍。

### 生成模型响应
｜Generate Model Responses

In [ ]:
def generate(prompt, max_length=4096, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

### 构造提示，可选地使用系统上下文和/或示例
｜Costruct Prompt, Optionally With System Context and/or Examples

In [ ]:
def construct_prompt_with_context(main_prompt, system_context="", conversation_examples=[]):
    """
    Constructs a complete structured prompt for a language model, including optional system context and conversation examples.

    This function compiles a prompt that can be directly used for generating responses from a language model. 
    It creates a structured format that begins with an optional system context message, appends a series of conversational 
    examples as prior interactions, and ends with the main user prompt. If no system context or conversation examples are provided,
    it will return only the main prompt.

    Parameters:
    - main_prompt (str): The core question or statement for the language model to respond to.
    - system_context (str, optional): Additional context or information about the scenario or environment. Defaults to an empty string.
    - conversation_examples (list of tuples, optional): Prior exchanges provided as context, where each tuple contains a user message 
      and a corresponding agent response. Defaults to an empty list.

    Returns:
    - str: A string formatted as a complete prompt ready for language model input. If no system context or examples are provided, returns the main prompt.

    Example usage:
    ```
    main_prompt = "I'm looking to improve my dialogue writing skills for my next short story. Any suggestions?"
    system_context = "User is an aspiring author seeking to enhance dialogue writing techniques."
    conversation_examples = [
        ("How can dialogue contribute to character development?", "Dialogue should reveal character traits and show personal growth over the story arc."),
        ("What are some common pitfalls in writing dialogue?", "Avoid exposition dumps in dialogue and make sure each character's voice is distinct.")
    ]

    full_prompt = construct_prompt_with_context(main_prompt, system_context, conversation_examples)
    print(full_prompt)
    ```
    """
    
    # Return the main prompt if no system context or conversation examples are provided
    if not system_context and not conversation_examples:
        return main_prompt

    # Start with the initial part of the prompt including the system context, if provided
    full_prompt = f"<s>[INST] <<SYS>>{system_context}<</SYS>>\n" if system_context else "<s>[INST]\n"

    # Add each example from the conversation_examples to the prompt
    for user_msg, agent_response in conversation_examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main user prompt at the end
    full_prompt += f"{main_prompt} [/INST]"

    return full_prompt

### LlamaChatbot 类
｜LlamaChatbot Class

In [ ]:
class LlamaChatbot:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
    - system_context (str): Contextual information to provide to the language model for all conversations.
    - conversation_history (list of tuples): Stores the history of the conversation, where each
      tuple contains a user message and the corresponding agent response.
    """

    def __init__(self, system_context):
        """
        Initializes a new instance of the LlamaChatbot class.

        Parameters:
        - system_context (str): A string that sets the initial context for the language model.
        """
        self.system_context = system_context
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history.

        Parameters:
        - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
        - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

### LlamaChatBotWithHistoryLimit 类
｜LlamaChatBotWithHistoryLimit Class

In [ ]:
class LlamaChatbotWithHistoryLimit:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
        - system_context (str): Contextual information to provide to the language model for all conversations.
        - conversation_history (list of tuples): Stores the history of the conversation, where each
          tuple contains a user message and the corresponding agent response.
        - tokenizer: The tokenizer used to tokenize the conversation for maintaining the history limit.
        - max_tokens (int): The maximum number of tokens allowed in the conversation history.
    """

    def __init__(self, system_context, tokenizer, max_tokens=2048):
        """
        Initializes a new instance of the LlamaChatbot class with a tokenizer and token limit.

        Parameters:
            - system_context (str): A string that sets the initial context for the language model.
            - tokenizer: The tokenizer used to process the input and output for the language model.
            - max_tokens (int): The maximum number of tokens to retain in the conversation history.
        """
        self.system_context = system_context
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history, ensuring that the history does not exceed the specified token limit.

        Parameters:
            - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
            - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        # Check and maintain the conversation history within the token limit
        self._trim_conversation_history()

        return agent_response

    def _trim_conversation_history(self):
        """
        Trims the conversation history to maintain the number of tokens below the specified limit.
        """
        # Concatenate the conversation history into a single string
        history_string = ''.join(user + agent for user, agent in self.conversation_history)
        
        # Calculate the number of tokens in the conversation history
        history_tokens = len(self.tokenizer.encode(history_string))

        # While the history exceeds the maximum token limit, remove the oldest items
        while history_tokens > self.max_tokens:
            # Always check if there's at least one item to pop
            if self.conversation_history:
                # Remove the oldest conversation tuple
                self.conversation_history.pop(0)
                # Recalculate the history string and its tokens
                history_string = ''.join(user + agent for user, agent in self.conversation_history)
                history_tokens = len(self.tokenizer.encode(history_string))
            else:
                # If the conversation history is empty, break out of the loop
                break

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

### 打印给定字符串中的令牌数
｜Print Number of Tokens in a Given String

In [ ]:
def print_token_count(text, tokenizer):
    """
    Calculate and return the number of tokens in a given text using a specified tokenizer.

    This function takes a string of text and a tokenizer. It uses the tokenizer to encode the text
    into tokens and then returns the count of these tokens.

    Parameters:
    - text (str): The input string to be tokenized.
    - tokenizer: A tokenizer instance capable of encoding text into tokens.

    Returns:
    - int: The number of tokens in the input text as determined by the tokenizer.
    """
    return len(tokenizer.encode(text))

### 连接对话历史记录
｜Concatenate Conversation History

In [ ]:
def concat_history(tuples_list):
    """
    Concatenates texts from a list of 2-tuples.

    Each tuple in the list is expected to contain two strings. The function
    will concatenate all the first elements followed by all the second elements
    in their respective order of appearance in the list.

    Parameters:
    - tuples_list (list of 2-tuples): A list where each element is a tuple of two strings.

    Returns:
    - str: A single string that is the result of concatenating all the texts from the tuples.

    Example usage:
    ```
    conversation_tuples = [
        ('Question 1', 'Answer 1'),
        ('Question 2', 'Answer 2'),
        ('Question 3', 'Answer 3')
    ]

    concatenated_text = concatenate_texts_from_tuples(conversation_tuples)
    print(concatenated_text)
    ```
    """
    # Concatenate all the first and second elements of the tuples
    return ''.join(question + response for question, response in tuples_list)

## 数据
｜Data

### 星辰自行车详细信息
｜Star Bikes Details

In [ ]:
bikes = [
    {
        "model": "Galaxy Rider",
        "type": "Mountain",
        "features": {
            "frame": "Aluminum alloy",
            "gears": "21-speed Shimano",
            "brakes": "Hydraulic disc",
            "tires": "27.5-inch all-terrain",
            "suspension": "Full, adjustable",
            "color": "Matte black with green accents"
        },
        "usps": ["Lightweight frame", "Quick gear shift", "Durable tires"],
        "price": 799.95,
        "internal_id": "GR2321",
        "weight": "15.3 kg",
        "manufacturer_location": "Taiwan"
    },
    {
        "model": "Nebula Navigator",
        "type": "Hybrid",
        "features": {
            "frame": "Carbon fiber",
            "gears": "18-speed Nexus",
            "brakes": "Mechanical disc",
            "tires": "26-inch city slick",
            "suspension": "Front only",
            "color": "Glossy white"
        },
        "usps": ["Sleek design", "Efficient on both roads and trails", "Ultra-lightweight"],
        "price": 649.99,
        "internal_id": "NN4120",
        "weight": "13.5 kg",
        "manufacturer_location": "Germany"
    },
    {
        "model": "Cosmic Comet",
        "type": "Road",
        "features": {
            "frame": "Titanium",
            "gears": "24-speed Campagnolo",
            "brakes": "Rim brakes",
            "tires": "700C road",
            "suspension": "None",
            "color": "Metallic blue"
        },
        "usps": ["Super aerodynamic", "High-speed performance", "Professional-grade components"],
        "price": 1199.50,
        "internal_id": "CC5678",
        "weight": "11 kg",
        "manufacturer_location": "Italy"
    }
]

## 自行车 AI 助手
｜Bikes AI Assistant

在本节中，我们将创建一个 AI 客户支持助手，以帮助潜在客户购买他们的下一辆星辰自行车。

我们将首先设置一个合适的**系统上下文（system context）** 并实例化一个聊天机器人实例。

In [ ]:
system_context = """
You are a friendly chatbot knowledgeable about bicycles. \
When asked about specific bike models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability in 50 words or less.
"""

chatbot = LlamaChatbot(system_context)

让我们要求模型告诉我们有关最新自行车的更多信息。

In [ ]:
print(chatbot.chat("Can you tell me about the latest models?"))

---

这还不错，但当然我们希望助手告诉我们有关星辰自行车的型号！

In [ ]:
chatbot.reset()

## 星辰自行车 AI 助手
｜Star Bikes AI Assistant

让我们创建一个新的聊天机器人，包括上面提供的`bikes`数据，以便它在对话期间参考。在以下**系统上下文** 中，我们为模型提供了一个**提示（cue）** ，让它始终以询问还能提供什么帮助来结束交换。这不仅对于 AI 助手来说是一个好主意，而且在实践中，可以防止模型无限期地继续，或者在只需要一个交换时尝试生成多个交换。

In [ ]:
system_context = f"""
You are a friendly chatbot knowledgeable about these bicycles from Star Bikes {bikes}. \
When asked about specific bike models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability in 50 words or less. \
You always end by asking what else you can help with.
"""

chatbot = LlamaChatbot(system_context)

In [ ]:
print(chatbot.chat("Can you tell me about the latest models?"))

---

这非常棒。让我们看看当被要求提供有关自行车的具体详细信息时，它会如何响应？

In [ ]:
print(chatbot.chat("How much do each of the models cost?"))

---

非常好。让我们看看它如何响应更模糊的查询。

In [ ]:
print(chatbot.chat("I am more intersted in biking around town."))

---

总的来说，我们的助手似乎已经表现得相当出色。

## 关于令牌数的考虑
｜Considerations About Number of Tokens

当我们将文本传递给像 LLaMA-2 这样的语言模型时，文本会被转换为**令牌（tokens）** ，即语言模型用于处理和生成文本的文本单元，例如单词或标点符号。

像 LLaMA-2 这样的语言模型具有内在的**令牌限制（token limit）** ，即它们在一个提示-响应周期中可以处理的令牌数量的固定上限。这种限制是由于它们的设计以及处理令牌所需的计算资源造成的。在我们的 LLaMA-2 模型的情况下，**令牌限制** 设置为`4096`个令牌。给定模型的令牌限制可以通过其文档获得，但在其固有限制内也可以控制。当使用像我们这样的`transformers`管道时，我们使用`max_length`参数控制**令牌限制** 。

内在的令牌限制或`max_length`参数（以较小者为准）决定了分配给*输入提示（input prompt）和模型输出（model's output）*的令牌总数。

由于我们没有清除上面聊天交换中的聊天历史记录，因此让我们看一下当前`chatbot`实例的`conversation_history`。

In [ ]:
print(chatbot.conversation_history)

---

为了支持获取所有这些字符串表示的**令牌** 数量，我们将使用上面定义的`concat_history`辅助函数来连接对话历史记录中的所有字符串。

In [ ]:
conv_history = concat_history(chatbot.conversation_history)

In [ ]:
print(conv_history)

---

现在，我们将使用上面定义的另一个辅助函数`print_token_count`来**标记化（tokenize）** 我们的对话历史记录字符串，使用 LLaMA-2 的**分词器（tokenizer）** （如上导入）。

In [ ]:
print_token_count(conv_history, tokenizer)

---

让我们看看与聊天机器人的额外交换如何逐渐增加对话历史记录中的**令牌** 数量。

In [ ]:
print(chatbot.chat("What kind of bike would be best if I'm on a budget?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("What's the next most expensive bike after the Galaxy Rider?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("Why is titanium so good for a frame?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("Do you remember where I said I was most interested in riding?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("Can you please summarize our conversation for me?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

---

为了结束此探索，我们将重置聊天机器人并再次打印令牌计数。

In [ ]:
chatbot.reset()

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

---

鉴于我们的聊天机器人实现通过在后续交换中将对话历史记录传递到提示中来存储先前的对话，因此，每次交换时，我们都在接近模型的**令牌限制** 。

如上所述，我们正在使用的模型的内在**令牌限制** 是`4096`，如果您查看上面`generate`函数的定义，我们会将`4096`作为`max_length`参数传递。因此，我们还没有接近**令牌限制** ，但是，我们应该考虑如何确保此硬限制不会在使用我们的聊天机器人时造成问题。

## 限制聊天历史记录
｜Limit Chat History

下面是一个修改后的聊天类`LlamaChatbotWithHistoryLimit`。它接受`max_tokens`参数，以及一个将用于跟踪对话历史记录中存在的令牌数量的分词器。

在对话历史记录将超过`max_tokens`的情况下，将调用`_trim_conversation_history`以弹出历史记录中最旧的对话，直到历史记录低于`max_tokens`。

In [ ]:
class LlamaChatbotWithHistoryLimit:
    """
    A chatbot interface for generating conversational responses using the LLaMA language model.

    Attributes:
        - system_context (str): Contextual information to provide to the language model for all conversations.
        - conversation_history (list of tuples): Stores the history of the conversation, where each
          tuple contains a user message and the corresponding agent response.
        - tokenizer: The tokenizer used to tokenize the conversation for maintaining the history limit.
        - max_tokens (int): The maximum number of tokens allowed in the conversation history.
    """

    def __init__(self, system_context, tokenizer, max_tokens=2048):
        """
        Initializes a new instance of the LlamaChatbot class with a tokenizer and token limit.

        Parameters:
            - system_context (str): A string that sets the initial context for the language model.
            - tokenizer: The tokenizer used to process the input and output for the language model.
            - max_tokens (int): The maximum number of tokens to retain in the conversation history.
        """
        self.system_context = system_context
        self.tokenizer = tokenizer
        self.max_tokens = max_tokens
        self.conversation_history = []  # Initializes the conversation history

    def chat(self, user_msg):
        """
        Generates a response from the chatbot based on the user's message.

        This method constructs a prompt with the current system context and conversation history,
        sends it to the language model, and then stores the new user message and model's response
        in the conversation history, ensuring that the history does not exceed the specified token limit.

        Parameters:
            - user_msg (str): The user's message to which the chatbot will respond.

        Returns:
            - str: The generated response from the chatbot.
        """
        # Generate the prompt using the conversation history and the new user message
        prompt = construct_prompt_with_context(user_msg, self.system_context, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        # Check and maintain the conversation history within the token limit
        self._trim_conversation_history()

        return agent_response

    def _trim_conversation_history(self):
        """
        Trims the conversation history to maintain the number of tokens below the specified limit.
        """
        # Concatenate the conversation history into a single string
        history_string = ''.join(user + agent for user, agent in self.conversation_history)
        
        # Calculate the number of tokens in the conversation history
        history_tokens = len(self.tokenizer.encode(history_string))

        # While the history exceeds the maximum token limit, remove the oldest items
        while history_tokens > self.max_tokens:
            # Always check if there's at least one item to pop
            if self.conversation_history:
                # Remove the oldest conversation tuple
                self.conversation_history.pop(0)
                # Recalculate the history string and its tokens
                history_string = ''.join(user + agent for user, agent in self.conversation_history)
                history_tokens = len(self.tokenizer.encode(history_string))
            else:
                # If the conversation history is empty, break out of the loop
                break

    def reset(self):
        """
        Resets the conversation history of the chatbot.

        This method clears the existing conversation history, effectively restarting the conversation.
        """
        # Clear conversation history
        self.conversation_history = []

让我们创建一个新的聊天机器人实例，这次使用`max_tokens`限制为`200`个令牌。

In [ ]:
system_context = f"""
You are a friendly chatbot knowledgeable about these bicycles from Star Bikes {bikes}. \
When asked about specific bike models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability in 50 words or less. \
You always end by asking what else you can help with.
"""

chatbot = LlamaChatbotWithHistoryLimit(system_context, tokenizer=tokenizer, max_tokens=200)

In [ ]:
print(chatbot.chat("Can you tell me about the latest models?"))

---

我们将运行更多几次交换，并跟踪对话历史记录中的令牌数量。请记住，我们已将`max_tokens`设置为`200`。

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("How much do each of the models cost?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

---

您可以看到令牌计数已减少到`96`，以防止我们超过指定的`200`个令牌限制。让我们观察更多几轮对话。

In [ ]:
print(chatbot.chat("I am more intersted in biking around town."))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
print(chatbot.chat("What kind of bike would be best if I'm on a budget?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

---

我们的聊天机器人已成功弹出早期的对话轮次以避免超过限制。

当然，作为这种故障安全行为的交换，我们牺牲了对话历史记录的完美保留。在这里我们可以看到，与之前不同，当我们要求总结迄今为止的对话时，我们只收到了我们最近几次交换的摘要。

In [ ]:
print(chatbot.chat("Can you summarize our conversation?"))

In [ ]:
print_token_count(concat_history(chatbot.conversation_history), tokenizer)

In [ ]:
chatbot.reset()

## 最终练习：为自己的虚构公司创建 AI 助手
｜Final Exercise: Create an AI Assistant for Your Own Fictitious Company

利用您迄今为止学到的所有知识，为一个您选择的虚构公司创建一个 AI 助手。您的工作将包括几个主要步骤。

1) 想出一个公司的创意，包括其名称和将要销售的产品。
2) 使用我们的 LLaMA-2 模型为您的公司将要销售的产品生成合成数据。请参阅上面的“星辰自行车详细信息”部分或`bikes`字典作为示例。如果您在生成合成 JSON 数据时遇到困难，请参阅笔记本*3-Review Analyst.ipynb*。
3) 创建 AI 助手，并为其提供您在上一步中生成的合成数据。您可以随意使用此笔记本中的`LlamaChatbotWithHistoryLimit`类。

## 关键概念回顾
｜Key Concept Review

本笔记本中介绍了以下关键概念：

- **令牌（Token）：** 语言模型用于处理的文本片段，例如单词或标点符号。
- **令牌限制（Token Limit）：** 语言模型在一个提示中可以处理的最大令牌数。
- **分词器（Tokenizer）：** 一种将文本转换为令牌以供语言模型理解的工具。

- **Token:** A piece of text, like a word or punctuation, used by language models for processing.
- **Token Limit:** The maximum number of tokens a language model can process in a single prompt.
- **Tokenizer:** A tool that converts text into tokens for language models to understand.

## 重新启动内核
｜Restart the Kernel

为了释放下一个笔记本的 GPU 内存，请运行以下单元格以重新启动内核。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)

# [5LOI DEEP LEARNING INSTITUE](https://5loi.com)

### [AI COMMUNITY](https://www.theforage.cn/community)

### [5LOI](https://5loi.com/about_loi)

![DLI Header](images/DLI_Header.png)